In [1]:
!pip install vertica_python
!pip install faker


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from time import time
from uuid import uuid4


import vertica_python

from faker import Faker

In [7]:
fake: Faker = Faker()

connection_info = {
    'host': 'localhost',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
}

connection = vertica_python.connect(**connection_info)
cursor = connection.cursor()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS player_progress (
        user_id UUID NOT NULL,
        movie_id UUID NOT NULL,
        event_dt DateTime NOT NULL,
        view_progress INTEGER NOT NULL,
        movie_duration INTEGER NOT NULL
    );
    """)

cursor.execute('select table_name from v_catalog.tables')
data = cursor.fetchall()[0]
print(data)


['player_progress']


In [11]:
connection_info = {
    'host': 'localhost',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'database': 'docker',
    'autocommit': True,
}

connection = vertica_python.connect(**connection_info)
cursor = connection.cursor()

BATCH_SIZE: int = 1000
BATCHES: int = 50
TOTAL_RECORDS: float = BATCH_SIZE * BATCHES


sql_query = \
    'INSERT INTO player_progress \
    (user_id, movie_id, event_dt, view_progress, movie_duration) VALUES (%s, %s, %s, %s, %s)'

start_time: float = time()
for _ in range(BATCHES):
    rows_to_insert = [
        (uuid4(),
         uuid4(),
         fake.date_time_between(start_date="-1y", end_date="now"),
         fake.random_int(min=0, max=1000),
         fake.random_int(min=1001, max=5000))
        for _ in range(BATCH_SIZE)
    ]
    
    cursor.executemany(sql_query, rows_to_insert)

insertion_time: float = time() - start_time
insertion_speed: float = round(TOTAL_RECORDS / insertion_time, 2)

print('Insertion speed: {:,} records/sec'.format(insertion_speed))

Insertion speed: 18,625.5 records/sec


In [12]:
get_rows_count_sql_query = 'SELECT COUNT(*) FROM player_progress'
cursor.execute(get_rows_count_sql_query)
rows_count = cursor.fetchall()[0][0]

FULL_DATA_SQL_QUERY = 'SELECT * FROM player_progress '
TRIES_COUNT: int = 5

start_time: float = time()
for _ in range(TRIES_COUNT):
    cursor.execute(FULL_DATA_SQL_QUERY)

selecting_time: float = time() - start_time
rows_per_second: float = round(rows_count / (selecting_time / TRIES_COUNT), 2)

print('Select speed: {:,} records/sec'.format(rows_per_second))

Select speed: 220,495.48 records/sec
